In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

In [2]:
import json

In [16]:
# Import API key
# from api_keys import audio_api

In [17]:
#create an empty list to populate with artist names
artist_names=[]

In [18]:
#generate random numbers in the range of available artist IDs
#used to populate artist_names list
random_nums = np.random.randint(low=100000, high=300000, size=20)
#print(ran_ids)

#test 
#random_nums = [112024, 0, 100000, 114364]

In [19]:
#create a base url
#example: https://theaudiodb.com/api/v1/json/1/artist.php?i=112024
id_url = "https://theaudiodb.com/api/v1/json/1/artist.php?i="

#create a loop that uses random number
for num in random_nums:
    unique_url = id_url + f'{num}'
    
    #call api url
    request = requests.get(unique_url)
    
    #for each attempt try to convert information to json
    try:
        #convert to json
        info = request.json()
    
    #exception json decode error
    #https://docs.python.org/3/tutorial/controlflow.html
    except json.JSONDecodeError:
        #end this iteration and continue new iteration of for loop
        continue
    
    #if json conversion successful then
    #try to create a variable for single artist to reference later
    try:
        artist = info['artists'][0]['strArtist']
        
    #TypeError exception if url responds with {'artists': None}
    except (TypeError):
        continue
        
    #if the artist is not already in the list then 
    if artist not in artist_names:
        
        #populate artist_names list using info(json)
        #response-> {'artists': [{'idArtist': '114364', 'strArtist': 'Beyoncé', ...
        #dictionary{'artists':['{dictionary}']}
        artist_names.append(info['artists'][0]['strArtist'])
    
    print(artist_names)

['Moritz Von Oswald Trio']
['Moritz Von Oswald Trio', 'Nina Hagen Band']
['Moritz Von Oswald Trio', 'Nina Hagen Band', 'GTA']
['Moritz Von Oswald Trio', 'Nina Hagen Band', 'GTA', 'Theme Park']
['Moritz Von Oswald Trio', 'Nina Hagen Band', 'GTA', 'Theme Park', 'Dark Fortress']
['Moritz Von Oswald Trio', 'Nina Hagen Band', 'GTA', 'Theme Park', 'Dark Fortress', 'Celeste Buckingham']
['Moritz Von Oswald Trio', 'Nina Hagen Band', 'GTA', 'Theme Park', 'Dark Fortress', 'Celeste Buckingham', 'Philippe Courbois']


In [20]:
#alternative to "if not in list then .append"
#convert list to set to remove redundant artists

#unique_artist_ids = set(artist_ids)
#print(unique_artist_ids)


In [21]:
#create loop to pull information from the database

#example url: http://theaudiodb.com/api/v1/json/1/search.php?s=coldplay

audio_data = {'artist':[],
              'style':[], 
              'genre':[]
             }

#loop over artists dictionary and add data to dictionary
for artist in artist_names:
    base_url = "http://theaudiodb.com/api/v1/json/1/search.php?s="
            #"http://theaudiodb.com/api/v1/json/1/search.php?s="
    try:
        req = requests.get(base_url + f'{artist}')
        data = req.json()
        #url = urllib3.urlopen(base_url + f'{artist}')
        #data = json.load(url)
        
        for item in data['artists']:
        #add values to dictionary
            #print(["strArtist"])
            #print(["strStyle"])
            #print(["strGenre"])
        
            audio_data['artist'].append(item["strArtist"])
            audio_data['style'].append(item["strStyle"])
            audio_data['genre'].append(item["strGenre"])
            
            #audio_data['artist'].append(item["artists"]["strArtist"])
            #audio_data['style'].append(item["artists"]["strStyle"])
            #audio_data['genre'].append(item["artists"]["strGenre"])
        
        #print(base_url + f'{artist}')
        
    except:
        pass

In [22]:
print(audio_data)

{'artist': ['Moritz Von Oswald Trio', 'Nina Hagen Band', 'GTA', 'Theme Park', 'Dark Fortress', 'Celeste Buckingham', 'Philippe Courbois'], 'style': ['', 'Rock/Pop', '', '', 'Metal', '', None], 'genre': ['', 'Hard Rock', 'House', 'Indie', 'Black Metal', 'Pop', '']}


In [23]:
audio_df = pd.DataFrame(audio_data)
print(audio_df)

                   artist     style        genre
0  Moritz Von Oswald Trio                       
1         Nina Hagen Band  Rock/Pop    Hard Rock
2                     GTA                  House
3              Theme Park                  Indie
4           Dark Fortress     Metal  Black Metal
5      Celeste Buckingham                    Pop
6       Philippe Courbois      None             


In [24]:
# https://www.youtube.com/watch?v=pxofwuWTs7c
# this turorital has been helpful

In [25]:
#possible questions
#location and genre
#year formed and genre

#pull artist, location, year formed, genre
